![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)
 
<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Flesson-plans&branch=anywidgets&urlpath=notebooks/lesson-plans/notebooks/plants/PhidgetFour_auto.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"></a>

# Watering plants with Phidgets (Manually)

This is a short notebook to collect and display data from "Phidgets" sensors around a plant and allow the user to water the plant.

## Overview

The purpose of this notebook is to collect data from certain hardware sensors called "Phidgets" which are set up to monitor the environment of a plant, and also to water the plant. The data can be viewed directly on the computer screen, and a water pump can be activated from a simple water pump.

- you need the Phidget VINT device, with three sensors attached
    - the temperature/humidity sensor (device type HUM1001_01)
    - the light sensor (device type LUX1000_0)
    - the moisture sensor (device type HUM1100)
- you will need a computer (Mac, Windows, Raspberry Pi) with a USB connection and internet access
- you will need the Chrome or Chromium Web browser (Safari, Firefox, and Edge will not work)

Follow these steps:

Assemble your Phidgets plant kit, see the following [link](https://www.phidgets.com/education/learn/projects/plant-kit/assemble/)

1. Setting up the interface
1. View the interface
1. Connecting the sensor devices
1. Watering your plant
1. Observing the results 


Run the cells in this notebook, one cell at a time. This will give you the chance to respond to any errors and fix things if necessary. Do not select "Run All." 

You should have a plant, with the sensors, water reservoir, and pump  nearby. This what the plant setup should look like. 

**Note: Keep it away from computer and electronic devices as much as possible.**

<div align="center">
<img src="images/plant-water.jpg" alt="A plant with sensors and water pump" width="400"/><br>
A plant with sensors and water pump.
</div>



Here is some more detail on the various devices we are using. 
<div align="center">
<img src="images/sensors.jpg" alt="Details of the pump and sensors" width="600"/><br>
1: Pump. 2: DC power supply. 3: Moisture probe. 4: Light sensor. 5: Humidity sensor. 6: VINT Hub
</div>

The notebook runs both Python code and Javascript (JS) code. It would be nice to do everything in Python, but it seems Javascript is necessary to communicate with the Phidgets in a Jupyter notebook. Fortunately, the Javascript code here is easy enough to read and you will not need to change it. 

***

## Step 1. Setting up the user interface

Our goal is to create a user interface that looks like this:


<div align="center">
<img src="images/WaterUI.jpg" alt="Image of the user interface" width="800"/><br>
A plant with sensors and water pump.
</div>

This UI includes a button to connect the Phidget, a second button to read the data, and a button to water the plant. It also holds four gauges to display values for temperature, humidity, soil moisture and light levels,  


## Step 1a. 

Let's set up the software widget called "PhidgetFive_read" that will to connect the Phidget hardware.

Click on `▶Run` in the following two cells.

In [ ]:
# Special libraries for the anywidgets
import pathlib
import traitlets
try:
    import anywidget
except:
    !pip install --user "anywidget[dev]"
    import anywidget

In [ ]:
## define the class
class PhidgetFive_read(anywidget.AnyWidget):
    _esm = pathlib.Path("ph5read.js")
    _css = pathlib.Path("ph4xx.css")
    # These 5 variables are used by the widget class. Do not delete. 
    hubPort = traitlets.Int(1).tag(sync=True)   # default is port 1 on Phidget Plant Kit
    temperature = traitlets.Float(0).tag(sync=True)
    humidity = traitlets.Float(0).tag(sync=True)
    moisture = traitlets.Float(0).tag(sync=True)
    luminance = traitlets.Float(0).tag(sync=True)
    pump = traitlets.Bool(False).tag(sync=True)

## create the PhidgetFour widget and name it pf
pf = PhidgetFive_read()
pf.hubPort = 1   ## The water pump is plugged into Port 1 (you can change this if you use another port)

# connect the Phidget widget to the gauges in the user interface
def updateTemperature(change):
    gauges.data[0]['value'] = change.new
def updateHumidity(change):
    gauges.data[1]['value'] = change.new
def updateMoisture(change):
    gauges.data[2]['value'] = change.new
def updateLuminance(change):
    gauges.data[3]['value'] = change.new
pf.observe(updateTemperature, names=["temperature"])
pf.observe(updateHumidity, names=["humidity"])
pf.observe(updateMoisture, names=["moisture"])
pf.observe(updateLuminance, names=["luminance"])

### Step 1b.

Next we set up the gauges and some buttons, using plotly and widgets. 

Click on `▶Run` in the following two cells.

In [ ]:
from IPython.display import display, Javascript
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ipywidgets as widgets
from time import sleep

In [ ]:
# the four sensor gauges
g_temp = go.Indicator(
    mode = "gauge+number",
    value = 20,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Temperature"},
    gauge = {'axis': {'range': [10, 40]}}
)

g_hum = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Humidity"},
    gauge = {'axis': {'range': [0, 100]}}
)

g_moist = go.Indicator(
    mode = "gauge+number",
    value = 0.5,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Moisture"},
    gauge = {'axis': {'range': [0, 1.0]}}
)

g_light = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Light Level"},
    gauge = {'axis': {'range': [0, 1000]}}
)

fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type' : 'domain'}, {'type' : 'domain'}],[{'type' : 'domain'}, {'type' : 'domain'}]],
    vertical_spacing = 0.35
)
fig.append_trace(g_temp, row=1, col=1)
fig.append_trace(g_hum, row=1, col=2)
fig.append_trace(g_moist, row=2, col=1)
fig.append_trace(g_light, row=2, col=2)

gauges = go.FigureWidget(fig)

# the data update button
data_button = widgets.Button(
    description='Update gauges',
    tooltip='Update data to gauges',
    disabled=False,
    button_style='success',
)

# the watering button
water_button = widgets.Button(
    description='Click to water',
    disabled=False,
    button_style='success',
    tooltip='Water for 2 seconds',
)
    

### Step 1c. Connecting the buttons

We connect the buttons in the user interface to functions that access the Phidget sensors.

Click on `▶Run` in the following cell

In [ ]:
## We connect the data button in the user interface to the Phidget widget named pf
def update_gauges(button):
    pf.send("read")

data_button.on_click(update_gauges)

## We connect the water button  the Phidget widget named pf
def water_plant(button):
    pf.send("pumpOn")
    button.button_style = 'info'
    sleep(2)
    pf.send("pumpOff")
    button.button_style = 'success'

water_button.on_click(water_plant)

***
## Step 2. Viewing the user interface

We can view the user interface with the display command. We assemble them using the HBox and VBox functions.

Click on `▶Run` in the following cell.

In [ ]:
dashboard = widgets.VBox([pf, gauges,widgets.HBox([data_button,water_button])])
display(dashboard)

***
## Step 3: Connecting the sensor devices

At this point, you should connect the Phidgets hardware to your computer. This includes six separate devices:
- the Phidget VINT device, attached to the computer's USB port
- the DC power supply, attached to Port 1 on the VINT device
- The water pump, attached to the DC power supply
- the soil moisture sensor, attached to the VINT
- the light sensor, attached to the VINT
- optionally, the combined temperature/humidity sensor, attached to the VINT

When you first attach the VINT device to the USB port, your computer may ask you whether you wish to connect to this USB device. Please answer "yes" to this security request. 


### Step 3a. Opening the USB connection

Steps connecting your USB device to your Phidget VINT device to your computer. 
1. The VINT must be plugged into your computer's USB port. 
1. Click on the 'Connect' button in the user interface above.
1. Follow the prompts to select the VINT device (a list appears that you should click on). This will pair the device with your computer. 

 ### 3b. Check.
 Clicking the "Connect" button should open a window that asks you to select the Phidget Hub.  The window looks like this:

<img src="images/Pconnect.png" width=300>
<div align="left">
    
Go ahead and slect the Hub, and click "Connect."

### 3c. Confirm the USB connection 

The status line below the 'Connect button' will tell you if the USB and devices are connected.

If all is working well, the status text will say "USB Connected." Click the button to update the date from the sensors.

If nothing happens, make sure you have the VINT hub connected to your computer's USB port. You should also have the sensors connected to the VINT hub. 

Click "Disconnect" and then "Connect" again. Try reading the data a second time. Sometimes the Phidgets need a few seconds to connect properly, so disconnecting and connecting again can help. 

***
## Step 4: Watering your plant

Now that everything is connected, we can view the user interface above. 

The gauges should be live, showing the values from the sensors whenever you click the update button.

Try the  "Click to Water" button. It should run the water pump for two seconds.


***
## Step 5. Observing the results
Once you water the plant, you should see the moisture level for the plant change.

The other data gauges are live as well, but you do need to click the update button to see the latest values. Can you see the temperature, humidity and light level changes throughout the day?

You will need to close this notebook and disconnect the Phidgets before using any other notebook with the Phidgets. 

### Emergency water cut off

If the water pump is running and not stopping, try running the following cell. Or just restart the kernel. 

In [ ]:
pf.send("pumpOff")

***
## 6. Final Step. Closing down the sensors

It is **important** to close down all the sensors when you are done, as otherwise they will keep busy forever. Also, this notebook will block other notebooks from using the same sensors.

You can disconnect by clicking the "Disconnect" button above. Or, simply close this notebook. 

#### Troubleshooting and/or not working?
If your are unable to read the data from the sensors, you can try the following:

1. Click disconnect and reconnect botton in this notebook
2. Check that the senors are connected to the VINT HUB Phidgets

## Conclusion

We have demonstrated the process of running a utility that retrieves Phidget sensor data and publishes it on the web. This operation involves a combination of Javascript and Python to interact with both the front-end and back-end components within the Jupyter notebook environment.

If you plan to collect data over an extended period, it's essential to have the Jupyter "Hub" software installed on your local computer. Please note that if you run this software on the [Callysto hub](https://hub.callysto.ca), it may automatically disconnect after approximately a day of continuous data collection.

You might also like to go back to the data collection notebooks: 
- [Collecting Data from Plants with Phidgets with Google Sheets](./plants-data-google-anywidget.ipynb)
- [Collecting Data from Plants with Phidgets with Ethercalc](./plants-data-ethercalc-anywidget.ipynb)

### Next Notebook
You may now go to the [next notebook](./plants-watering-auto-anywidget.ipynb), which automates the watering of your plant. The computer will monitor the moisture of the soil every day and decide whether to pump in more water. 

***
For additional support or inquiries related to this notebook, feel free to reach out to M. Lamoureux or Mary Grant, who serve as Callysto ambassadors. They can provide further details and assistance.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)